# FASA Week 9 Strategy - Due Wednesday 11:59 PM EST

Analysis notebook for FASA bid planning using `mrt_fasa_targets` and `mrt_my_roster_droppable`.

In [ ]:
# Imports
from pathlib import Path

import duckdb
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Set plotting style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Connect to data sources

# Determine repo root (parent of notebooks dir)
repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
duckdb_path = repo_root / 'dbt' / 'ff_data_transform' / 'target' / 'dev.duckdb'
data_root = repo_root / 'data' / 'raw'

print(f"Repo root: {repo_root}")
print(f"DuckDB: {duckdb_path}")
print(f"Data root: {data_root}")

# Load FASA targets directly from Parquet (external table)
# Using dynamic date selection - reads latest snapshot available
fasa_mart_dir = data_root / 'marts' / 'mrt_fasa_targets'
if fasa_mart_dir.exists():
    # Find all date partitions and use the most recent
    date_dirs = sorted([d for d in fasa_mart_dir.glob('dt=*') if d.is_dir()], reverse=True)
    if date_dirs:
        latest_fasa_path = date_dirs[0] / 'data.parquet'
        fasa_targets = pd.read_parquet(latest_fasa_path)
        print(f"✅ Loaded {len(fasa_targets)} FASA targets from {date_dirs[0].name}")
    else:
        print(f"❌ No date partitions found in {fasa_mart_dir}")
        fasa_targets = pd.DataFrame()
else:
    print(f"❌ FASA targets directory not found at: {fasa_mart_dir}")
    fasa_targets = pd.DataFrame()

# Load my roster droppable (also external table)
# Using dynamic date selection - reads latest snapshot available
droppable_mart_dir = data_root / 'marts' / 'mrt_my_roster_droppable'
if droppable_mart_dir.exists():
    # Find all date partitions and use the most recent
    date_dirs = sorted([d for d in droppable_mart_dir.glob('dt=*') if d.is_dir()], reverse=True)
    if date_dirs:
        latest_droppable_path = date_dirs[0] / 'data.parquet'
        my_roster_droppable = pd.read_parquet(latest_droppable_path)
        print(f"✅ Loaded {len(my_roster_droppable)} roster players from {date_dirs[0].name}")
    else:
        print(f"❌ No date partitions found in {droppable_mart_dir}")
        my_roster_droppable = pd.DataFrame()
else:
    print(f"❌ Roster droppable directory not found at: {droppable_mart_dir}")
    my_roster_droppable = pd.DataFrame()

# Load cap situation from DuckDB (regular table, not external)
try:
    conn = duckdb.connect(str(duckdb_path), read_only=True)
    cap_situation = conn.execute("""
        SELECT *
        FROM main.mrt_cap_situation
        WHERE owner_name = 'Jason'
    """).df()
    conn.close()
    print(f"✅ Loaded {len(cap_situation)} years of cap data from DuckDB")
except Exception as e:
    print(f"⚠️  Cap situation not available: {e}")
    # Use hardcoded values from Task 1.1 sprint plan
    cap_situation = pd.DataFrame([
        {'season': 2025, 'cap_space_available': 80},
        {'season': 2026, 'cap_space_available': 80},
        {'season': 2027, 'cap_space_available': 158},
        {'season': 2028, 'cap_space_available': 183},
        {'season': 2029, 'cap_space_available': 250}
    ])
    print("Using hardcoded cap values from Task 1.1")

print(f"\n{'='*60}")
print("DATA LOAD SUMMARY")
print(f"{'='*60}")
print(f"FASA targets: {len(fasa_targets)}")
print(f"Roster players: {len(my_roster_droppable)}")
print(f"Cap situation: {len(cap_situation)} years")

## 1. My Cap Situation

In [ ]:
# Display cap situation
print("Current Cap Space:")
cap_col = 'cap_space_available' if 'cap_space_available' in cap_situation.columns else 'cap_space'
print(cap_situation[['season', cap_col]].to_string(index=False))

# Get current year cap
cap_2025 = cap_situation[cap_situation['season'] == 2025]
if len(cap_2025) > 0:
    current_cap = cap_2025[cap_col].iloc[0]
else:
    current_cap = 80  # Fallback
    print("\nWarning: No 2025 cap data found, using hardcoded value")

print(f"\n2025 Available Cap: ${current_cap}")

In [ ]:
# Cap Space Calculator with Year-by-Year Breakdown
# NOTE: cap_space_freed = current_year_cap_hit - dead_cap_if_cut_now (net cap savings for 2025)


def calculate_cap_if_drop(player_name):
    """Calculate cap space if a player is dropped.
    
    Shows:
    - 2025 cap impact (immediate savings)
    - Year-by-year breakdown showing cap hit vs dead cap vs net savings
    
    Cap Space Freed Calculation:
    cap_space_freed = current_year_cap_hit - dead_cap_if_cut_now
    
    This represents the NET cap savings in 2025 after accounting for dead cap obligation.
    """
    player = my_roster_droppable[my_roster_droppable['player_name'] == player_name]
    
    if len(player) == 0:
        return {'error': f'Player {player_name} not found'}
    
    player = player.iloc[0]
    cap_freed = player['cap_space_freed']
    new_cap_space = current_cap + cap_freed
    
    result = {
        'player': player_name,
        'current_cap': current_cap,
        'cap_freed': cap_freed,
        'dead_cap': player['dead_cap_if_cut_now'],
        'new_cap_space': new_cap_space
    }
    
    # Try to load year-by-year contract breakdown from DuckDB
    try:
        conn = duckdb.connect(str(duckdb_path), read_only=True)
        
        # Get player's multi-year contract obligations
        contract_years = conn.execute("""
            SELECT 
                c.obligation_year,
                c.cap_hit as current_cap_hit,
                c.years_remaining_at_sign,
                cls.dead_cap_pct,
                ROUND(c.cap_hit * cls.dead_cap_pct, 2) as dead_cap_this_year,
                ROUND(c.cap_hit * (1 - cls.dead_cap_pct), 2) as net_savings_this_year
            FROM main.stg_sheets__contracts_active c
            LEFT JOIN main.dim_cut_liability_schedule cls
                ON c.years_remaining_at_sign = cls.years_remaining_at_sign
                AND (c.obligation_year - c.contract_year + 1) = cls.year_into_contract
            WHERE c.player_name = ?
                AND c.gm_full_name = 'Jason Shaffer'
            ORDER BY c.obligation_year
        """, [player_name]).df()
        
        conn.close()
        
        if len(contract_years) > 0:
            print(f"\n📊 Year-by-Year Cap Impact for {player_name}:")
            print("="*70)
            print(f"{'Year':<8} {'Current Hit':<15} {'Dead Cap':<15} {'Net Savings':<15}")
            print("-"*70)
            
            for _, row in contract_years.iterrows():
                year = int(row['obligation_year'])
                current_hit = float(row['current_cap_hit'])
                dead_cap = float(row['dead_cap_this_year']) if pd.notna(row['dead_cap_this_year']) else 0.0
                net_savings = float(row['net_savings_this_year']) if pd.notna(row['net_savings_this_year']) else current_hit
                
                print(f"{year:<8} ${current_hit:<14.2f} ${dead_cap:<14.2f} ${net_savings:<14.2f}")
            
            print("="*70)
            print(f"2025 Net Cap Freed: ${cap_freed:.2f} (added to current ${current_cap} = ${new_cap_space:.2f})")
            
    except Exception as e:
        print(f"⚠️  Could not load year-by-year breakdown: {e}")
        print(f"Using summary calculation: ${cap_freed:.2f} net cap freed in 2025")
    
    return result


# Example usage
if len(my_roster_droppable) > 0:
    example_player = my_roster_droppable.iloc[0]['player_name']
    print(f"Example: Drop {example_player}")
    result = calculate_cap_if_drop(example_player)
    print(f"\nSummary: {result}")

## 2. Top FASA Targets by Position

### 2.1 RB Targets (Priority #1)

In [ ]:
# Top 10 RBs Available

# NOTE: fantasy_ppg_last_4 and projected_ppg_ros calculations:
# - Actual PPG: Calculated from nflverse NFL stats using league's scoring rules (dim_scoring_rule)
# - Pipeline: nflverse → fact_player_stats → mrt_fantasy_actuals_weekly
# - Scoring: Half-PPR (0.5 per rec) + standard rushing/receiving + IDP scoring
# - Minor variance from Sleeper expected due to:
#   * Different stat providers (nflverse vs Sleeper's proprietary source)
#   * Different calculation timing/snapshots
#   * Rounding differences
# - Our calculations are transparent and auditable; discrepancies of <0.2 PPG are typical

rb_targets = fasa_targets[fasa_targets['position'] == 'RB'].head(10)

display_cols = [
    'priority_rank_at_position',
    'player_name',
    'nfl_team',
    'fantasy_ppg_last_4',
    'projected_ppg_ros',
    'opportunity_share_l4',
    'value_score',
    'suggested_bid_1yr',
    'suggested_bid_2yr',
    'bid_confidence'
]

print("Top 10 RB Targets:")
display(rb_targets[display_cols])

In [ ]:
# RB Value Matrix Visualization
if len(rb_targets) > 0:
    plt.figure(figsize=(12, 8))
    plt.scatter(rb_targets['suggested_bid_1yr'], rb_targets['projected_ppg_ros'], 
                s=100, alpha=0.6, c=rb_targets['value_score'], cmap='RdYlGn')
    plt.colorbar(label='Value Score')
    plt.xlabel('Suggested Bid ($)', fontsize=12)
    plt.ylabel('Projected PPG ROS', fontsize=12)
    plt.title('RB Value Matrix: Projected Performance vs Bid Cost', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    # Annotate players
    for _, row in rb_targets.iterrows():
        plt.annotate(row['player_name'], 
                    (row['suggested_bid_1yr'], row['projected_ppg_ros']),
                    fontsize=9, alpha=0.8, 
                    xytext=(5, 5), textcoords='offset points')
    plt.tight_layout()
    plt.show()

### 2.2 WR Targets (Priority #2)

In [ ]:
# Top 15 WRs Available

# NOTE: Bid Logic and Value Scores:
# - suggested_bid uses simple $/point formula:
#   * RB: $1 per 10 projected points ROS
#   * WR: $1 per 12 projected points ROS  
#   * TE: $1 per 15 projected points ROS
# - value_score is a composite metric (0-100) weighing:
#   * 40% projections (normalized by position)
#   * 25% opportunity share (snap/target share)
#   * 20% efficiency (YPC, YPR, catch rate)
#   * 15% market value (inverted KTC rank)
# - Players with SAME bid but DIFFERENT value scores = potential bargains
#   (high opportunity/efficiency not fully captured by projections)
# - Future enhancement: ML-based bids incorporating value_score (Sprint 2+)

wr_targets = fasa_targets[fasa_targets['position'] == 'WR'].head(15)

print("Top 15 WR Targets:")
display(wr_targets[display_cols])

In [ ]:
# WR Value Matrix Visualization
if len(wr_targets) > 0:
    plt.figure(figsize=(12, 8))
    plt.scatter(wr_targets['suggested_bid_1yr'], wr_targets['projected_ppg_ros'], 
                s=100, alpha=0.6, c=wr_targets['value_score'], cmap='RdYlGn')
    plt.colorbar(label='Value Score')
    plt.xlabel('Suggested Bid ($)', fontsize=12)
    plt.ylabel('Projected PPG ROS', fontsize=12)
    plt.title('WR Value Matrix: Projected Performance vs Bid Cost', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    # Annotate players
    for _, row in wr_targets.iterrows():
        plt.annotate(row['player_name'], 
                    (row['suggested_bid_1yr'], row['projected_ppg_ros']),
                    fontsize=9, alpha=0.8,
                    xytext=(5, 5), textcoords='offset points')
    plt.tight_layout()
    plt.show()

### 2.3 TE Targets (Priority #3)

In [ ]:
# Top 8 TEs Available
te_targets = fasa_targets[fasa_targets['position'] == 'TE'].head(8)

print("Top 8 TE Targets:")
display(te_targets[display_cols])

In [ ]:
# TE Value Matrix Visualization
if len(te_targets) > 0:
    plt.figure(figsize=(12, 8))
    plt.scatter(te_targets['suggested_bid_1yr'], te_targets['projected_ppg_ros'], 
                s=100, alpha=0.6, c=te_targets['value_score'], cmap='RdYlGn')
    plt.colorbar(label='Value Score')
    plt.xlabel('Suggested Bid ($)', fontsize=12)
    plt.ylabel('Projected PPG ROS', fontsize=12)
    plt.title('TE Value Matrix: Projected Performance vs Bid Cost', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    # Annotate players
    for _, row in te_targets.iterrows():
        plt.annotate(row['player_name'], 
                    (row['suggested_bid_1yr'], row['projected_ppg_ros']),
                    fontsize=9, alpha=0.8,
                    xytext=(5, 5), textcoords='offset points')
    plt.tight_layout()
    plt.show()

## 3. Bidding Strategy Matrix

In [ ]:
# RB Bidding Strategy
print("RB STRATEGY (Position of Need):\n")

if len(rb_targets) >= 3:
    rb_strategy = []
    for i, (_, player) in enumerate(rb_targets.head(3).iterrows(), 1):
        rb_strategy.append({
            'Tier': f'RB{i}',
            'Player': player['player_name'],
            'Team': player['nfl_team'],
            'Proj PPG': f"{player['projected_ppg_ros']:.1f}",
            'Bid (1yr)': f"${player['suggested_bid_1yr']:.0f}",
            'Bid (2yr)': f"${player['suggested_bid_2yr']:.0f}" if not pd.isna(player['suggested_bid_2yr']) else "N/A",
            'Confidence': player['bid_confidence'],
            'Logic': f"Value score: {player['value_score']:.0f}, Opp%: {player['opportunity_share_l4']*100:.1f}%"
        })
    
    rb_strategy_df = pd.DataFrame(rb_strategy)
    display(rb_strategy_df)
    
    print("\n⚠️  SEALED BID FORMAT - Submit all bids at once with priority:")
    print("\nRECOMMENDED BIDS (Priority Order):")
    print(f"  Priority 1: {rb_strategy_df.iloc[0]['Player']} - ${rb_targets.iloc[0]['suggested_bid_1yr']:.0f}/1yr")
    if len(rb_targets) > 1:
        print(f"  Priority 2: {rb_strategy_df.iloc[1]['Player']} - ${rb_targets.iloc[1]['suggested_bid_1yr']:.0f}/1yr")
    if len(rb_targets) > 2:
        print(f"  Priority 3: {rb_strategy_df.iloc[2]['Player']} - ${rb_targets.iloc[2]['suggested_bid_1yr']:.0f}/1yr")
    
    print("\nCONTINGENCY RULES:")
    print("  - If I win Priority 1, cancel Priority 2 & 3")
    print("  - If I lose Priority 1, proceed with Priority 2")
    print("  - Maximum: Acquire 1 RB this week")

In [ ]:
# WR Bidding Strategy
print("\nWR STRATEGY:\n")

if len(wr_targets) >= 3:
    wr_strategy = []
    for i, (_, player) in enumerate(wr_targets.head(3).iterrows(), 1):
        wr_strategy.append({
            'Tier': f'WR{i}',
            'Player': player['player_name'],
            'Team': player['nfl_team'],
            'Proj PPG': f"{player['projected_ppg_ros']:.1f}",
            'Bid (1yr)': f"${player['suggested_bid_1yr']:.0f}",
            'Bid (2yr)': f"${player['suggested_bid_2yr']:.0f}",
            'Confidence': player['bid_confidence']
        })
    
    wr_strategy_df = pd.DataFrame(wr_strategy)
    display(wr_strategy_df)

In [ ]:
# TE Bidding Strategy
print("\nTE STRATEGY:\n")

if len(te_targets) >= 2:
    te_strategy = []
    for i, (_, player) in enumerate(te_targets.head(2).iterrows(), 1):
        te_strategy.append({
            'Tier': f'TE{i}',
            'Player': player['player_name'],
            'Team': player['nfl_team'],
            'Proj PPG': f"{player['projected_ppg_ros']:.1f}",
            'Bid (1yr)': f"${player['suggested_bid_1yr']:.0f}",
            'Bid (2yr)': f"${player['suggested_bid_2yr']:.0f}",
            'Confidence': player['bid_confidence']
        })
    
    te_strategy_df = pd.DataFrame(te_strategy)
    display(te_strategy_df)

## 4. Drop Scenarios (if cap space needed)

In [ ]:
# Load league roster rules and check constraints
try:
    conn = duckdb.connect(str(duckdb_path), read_only=True)
    
    # Get current roster rules
    league_rules = conn.execute("""
        SELECT *
        FROM dim_league_rules
        WHERE is_current = true
    """).df()
    
    # Get my current ACTIVE roster composition (exclude TAXI, IDP TAXI, IR)
    # NOTE: FASA acquisitions go to active roster only, not eligible for TAXI/IR initially
    my_active_roster_composition = conn.execute("""
        SELECT 
            d.position,
            COUNT(*) as player_count
        FROM main.stg_sheets__contracts_active c
        INNER JOIN main.dim_player d ON c.player_id = d.player_id
        WHERE c.gm_full_name = 'Jason Shaffer'
          AND c.obligation_year = 2025
          AND c.roster_slot NOT IN ('TAXI', 'IDP TAXI', 'IR')
        GROUP BY d.position
        ORDER BY player_count DESC
    """).df()
    
    # Get total roster (all slots) for reference
    my_total_roster = conn.execute("""
        SELECT COUNT(*) as total_count
        FROM main.stg_sheets__contracts_active c
        WHERE c.gm_full_name = 'Jason Shaffer'
          AND c.obligation_year = 2025
    """).df()
    
    conn.close()
    
    # Calculate constraints
    roster_active_limit = int(league_rules.iloc[0]['roster_active_limit'])
    roster_taxi_limit = int(league_rules.iloc[0]['roster_taxi_limit'])
    roster_ir_limit = int(league_rules.iloc[0]['roster_ir_limit'])
    roster_total_defined = int(league_rules.iloc[0]['roster_total_defined_slots'])
    
    current_active_roster_size = int(my_active_roster_composition['player_count'].sum())
    current_total_roster_size = int(my_total_roster.iloc[0]['total_count'])
    active_roster_spots_available = roster_active_limit - current_active_roster_size
    
    print("⚙️  ROSTER SIZE CONSTRAINTS:")
    print("="*60)
    print(f"Total roster: {current_total_roster_size}/{roster_total_defined} total slots")
    print(f"  • Active roster: {current_active_roster_size}/{roster_active_limit} slots")
    print(f"  • TAXI/IDP TAXI: {roster_taxi_limit} slots")
    print(f"  • IR: {roster_ir_limit} slots")
    print(f"Available ACTIVE spots: {active_roster_spots_available}")
    print("\n⚠️  NOTE: FASA pickups go to ACTIVE roster, not TAXI/IR")
    
    print("\n\nActive Roster by Position:")
    display(my_active_roster_composition)
    
    # Assess cap needs for top bid
    if len(rb_targets) > 0:
        top_rb_bid = rb_targets.iloc[0]['suggested_bid_1yr']
        cap_needed = top_rb_bid if pd.notna(top_rb_bid) else 0
        cap_gap = current_cap - cap_needed
        
        print("\n💰 CAP SPACE ANALYSIS:")
        print(f"Current cap: ${current_cap}")
        print(f"Top bid (RB1): ${cap_needed:.0f}")
        print(f"Remaining after bid: ${cap_gap:.0f}")
        
        # Determine if drop is needed
        need_roster_spot = active_roster_spots_available <= 0
        need_cap_space = cap_gap < 0
        
        print("\n✅ ACTION REQUIRED:")
        if need_roster_spot and need_cap_space:
            print(f"   ⚠️  MUST DROP: Need active roster spot AND ${abs(cap_gap):.0f} cap space")
        elif need_roster_spot:
            print(f"   ⚠️  MUST DROP: Active roster full ({current_active_roster_size}/{roster_active_limit})")
        elif need_cap_space:
            print(f"   ⚠️  MUST DROP: Need ${abs(cap_gap):.0f} cap space")
        else:
            print(f"   ✅ NO DROP REQUIRED: Have {active_roster_spots_available} active roster spot(s) and sufficient cap")
            
except Exception as e:
    print(f"⚠️  Could not load roster rules: {e}")
    print("Using fallback analysis...")
    
    if len(rb_targets) > 0:
        top_rb_bid = rb_targets.iloc[0]['suggested_bid_1yr']
        cap_needed = top_rb_bid if pd.notna(top_rb_bid) else 0
        cap_gap = current_cap - cap_needed
        
        print(f"Current Cap: ${current_cap}")
        print(f"Needed for RB1 bid: ${cap_needed:.0f}")
        print(f"Gap: ${cap_gap:.0f}")
        
        if cap_gap < 0:
            print(f"\n⚠️  NEED TO CREATE ${abs(cap_gap):.0f} IN CAP SPACE\n")
        else:
            print("\n✅ SUFFICIENT CAP SPACE\n")

In [ ]:
# Top Drop Candidates
print("Top 5 Drop Candidates:\n")

drop_candidates = my_roster_droppable.head(5)

drop_cols = [
    'player_name',
    'position',
    'current_year_cap_hit',
    'dead_cap_if_cut_now',
    'cap_space_freed',
    'projected_ppg_ros',
    'droppable_score',
    'drop_recommendation'
]

display(drop_candidates[drop_cols])

In [ ]:
# Scenario Analysis
print("\nDrop Scenario Analysis:\n")

if len(drop_candidates) > 0 and len(rb_targets) > 0:
    top_rb_ppg = rb_targets.iloc[0]['projected_ppg_ros']
    
    scenarios = []
    for _, player in drop_candidates.head(3).iterrows():
        cap_freed = player['cap_space_freed']
        dead_cap = player['dead_cap_if_cut_now']
        net_benefit = cap_freed - dead_cap
        value_lost = player['projected_ppg_ros']
        ppg_upgrade = top_rb_ppg - value_lost
        worth_it = "✅ YES" if ppg_upgrade > 3 else "⚠️ MARGINAL" if ppg_upgrade > 1 else "❌ NO"
        
        scenarios.append({
            'If I drop': player['player_name'],
            'Cap Freed': f"${cap_freed:.0f}",
            'Dead Cap': f"${dead_cap:.0f}",
            'Net Benefit': f"${net_benefit:.0f}",
            'Value Lost (PPG)': f"{value_lost:.1f}",
            'PPG Upgrade': f"+{ppg_upgrade:.1f}",
            'Worth It?': worth_it
        })
    
    scenarios_df = pd.DataFrame(scenarios)
    display(scenarios_df)

## 5. Position Depth Analysis

In [ ]:
# My Current RB Depth
my_rbs = my_roster_droppable[
    my_roster_droppable['position'] == 'RB'
].sort_values('projected_ppg_ros', ascending=False)

print("My Current RB Depth:")
display(my_rbs[['player_name', 'projected_ppg_ros', 'current_year_cap_hit']].head(5))

# Calculate depth metrics
if len(my_rbs) >= 3:
    my_rb_ppg = my_rbs['projected_ppg_ros'].head(3).tolist()
    league_median_rb_ppg = [12.5, 8.7, 4.5]  # Approximate league medians
    
    print(f"\nMy RB1: {my_rb_ppg[0]:.1f} PPG (League median: {league_median_rb_ppg[0]} PPG)")
    print(f"My RB2: {my_rb_ppg[1]:.1f} PPG (League median: {league_median_rb_ppg[1]} PPG)")
    print(f"My RB3: {my_rb_ppg[2]:.1f} PPG (League median: {league_median_rb_ppg[2]} PPG)")

In [ ]:
# Visualize RB depth comparison
if len(my_rbs) >= 3:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    x = ['RB1', 'RB2', 'RB3']
    width = 0.35
    x_pos = range(len(x))
    
    ax.bar([p - width/2 for p in x_pos], my_rb_ppg, width, label='My Team', alpha=0.8, color='steelblue')
    ax.bar([p + width/2 for p in x_pos], league_median_rb_ppg, width, label='League Median', alpha=0.8, color='coral')
    
    ax.set_ylabel('PPG', fontsize=12)
    ax.set_xlabel('Position', fontsize=12)
    ax.set_title('My RB Depth vs League Median', fontsize=14, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(x)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# FLEX Performance Impact & ROI Calculation
if len(my_rbs) >= 2 and len(rb_targets) > 0:
    current_flex_ppg = my_rbs.iloc[1]['projected_ppg_ros']  # RB2 currently in FLEX
    top_rb_ppg = rb_targets.iloc[0]['projected_ppg_ros']
    new_flex_ppg = my_rbs.iloc[0]['projected_ppg_ros']  # Current RB1 would move to FLEX
    
    ppg_gain = new_flex_ppg - current_flex_ppg
    weeks_remaining = 9
    total_points_gain = ppg_gain * weeks_remaining
    
    bid_cost = rb_targets.iloc[0]['suggested_bid_2yr']
    cost_per_point = bid_cost / total_points_gain if total_points_gain > 0 else 0
    league_median_cpp = 1.20  # Approximate
    
    print("FLEX PERFORMANCE IMPACT:\n")
    print(f"Current FLEX avg: {current_flex_ppg:.1f} PPG ({my_rbs.iloc[1]['player_name']})")
    print(f"If I add RB1 target (proj {top_rb_ppg:.1f} PPG), new FLEX: {new_flex_ppg:.1f} PPG ({my_rbs.iloc[0]['player_name']})")
    print(f"Expected gain: +{ppg_gain:.1f} PPG/week × {weeks_remaining} weeks = +{total_points_gain:.0f} total points")
    
    print("\nROI CALCULATION:\n")
    print(f"Bid cost: ${bid_cost:.0f}")
    print(f"Expected points gained: +{total_points_gain:.0f} points ROS")
    print(f"Cost per point: ${cost_per_point:.2f}/point")
    print(f"League median cost/point: ${league_median_cpp:.2f}/point")
    
    if cost_per_point < league_median_cpp:
        print(f"\n✅ GOOD VALUE (${(league_median_cpp - cost_per_point):.2f}/point better than median)")
    else:
        print(f"\n⚠️ EXPENSIVE (${(cost_per_point - league_median_cpp):.2f}/point worse than median)")

## 6. Final Recommendation

In [ ]:
# Generate sealed bid submission sheet
print("="*80)
print("SEALED BID SUBMISSION SHEET - Week 9 FASA")
print("="*80)

# Build bid matrix with all positions
bid_matrix = []

# Top RBs
for i, (_, player) in enumerate(rb_targets.head(5).iterrows(), 1):
    bid_1yr = player['suggested_bid_1yr']
    bid_matrix.append({
        'Priority': i,
        'Position': 'RB',
        'Player': player['player_name'],
        'Team': player['nfl_team'],
        'Bid_1yr': f"${bid_1yr:.0f}" if pd.notna(bid_1yr) else "$0",
        'Bid_2yr': f"${player['suggested_bid_2yr']:.0f}" if pd.notna(player['suggested_bid_2yr']) else "N/A",
        'Proj_PPG': f"{player['projected_ppg_ros']:.1f}",
        'Value': int(player['value_score']),
        'Confidence': player['bid_confidence']
    })

# Top WRs (if competitive value)
for i, (_, player) in enumerate(wr_targets.head(3).iterrows(), len(bid_matrix) + 1):
    if player['value_score'] > 40:  # Only include if decent value
        bid_1yr = player['suggested_bid_1yr']
        bid_matrix.append({
            'Priority': i,
            'Position': 'WR',
            'Player': player['player_name'],
            'Team': player['nfl_team'],
            'Bid_1yr': f"${bid_1yr:.0f}" if pd.notna(bid_1yr) else "$0",
            'Bid_2yr': f"${player['suggested_bid_2yr']:.0f}" if pd.notna(player['suggested_bid_2yr']) else "N/A",
            'Proj_PPG': f"{player['projected_ppg_ros']:.1f}",
            'Value': int(player['value_score']),
            'Confidence': player['bid_confidence']
        })

# Top TE (if competitive value)
if len(te_targets) > 0 and te_targets.iloc[0]['value_score'] > 40:
    player = te_targets.iloc[0]
    bid_1yr = player['suggested_bid_1yr']
    bid_matrix.append({
        'Priority': len(bid_matrix) + 1,
        'Position': 'TE',
        'Player': player['player_name'],
        'Team': player['nfl_team'],
        'Bid_1yr': f"${bid_1yr:.0f}" if pd.notna(bid_1yr) else "$0",
        'Bid_2yr': f"${player['suggested_bid_2yr']:.0f}" if pd.notna(player['suggested_bid_2yr']) else "N/A",
        'Proj_PPG': f"{player['projected_ppg_ros']:.1f}",
        'Value': int(player['value_score']),
        'Confidence': player['bid_confidence']
    })

bid_df = pd.DataFrame(bid_matrix)

print("\n📋 COMPLETE BID MATRIX (Ranked by Value):\n")
display(bid_df)

print("\n💡 SEALED BID INSTRUCTIONS:")
print("=" * 80)
print("1. Submit ALL bids above to commissioner in PRIORITY ORDER")
print("2. Commissioner processes in priority sequence:")
print("   - If Priority 1 wins → Cancel all lower priorities")
print("   - If Priority 1 loses → Process Priority 2")
print("   - Continue until one acquisition or all bids exhausted")
print("3. Maximum acquisitions this week: 1 player")
print("\n⚠️  Cap Constraint Check:")
print(f"   Current cap space: ${current_cap}")
print(f"   Highest bid: {bid_df.iloc[0]['Bid_1yr']} (Priority 1: {bid_df.iloc[0]['Player']})")
if current_cap >= float(bid_df.iloc[0]['Bid_1yr'].replace('$', '')):
    print("   ✅ SUFFICIENT CAP SPACE for all bids")
else:
    print("   ⚠️  WARNING: May need to drop player if winning bid")

print("\n🎯 TOP RECOMMENDATION:")
print(f"   Best value: {bid_df.iloc[0]['Player']} ({bid_df.iloc[0]['Position']})")
print(f"   Value score: {bid_df.iloc[0]['Value']} | Projected: {bid_df.iloc[0]['Proj_PPG']} PPG")
print(f"   Confidence: {bid_df.iloc[0]['Confidence']}")

print("\n" + "="*80)

In [ ]:
# Summary statistics
print("\n📊 SUMMARY STATISTICS:\n")
print(f"Total FASA targets analyzed: {len(fasa_targets)}")
print(f"RB targets: {len(fasa_targets[fasa_targets['position'] == 'RB'])}")
print(f"WR targets: {len(fasa_targets[fasa_targets['position'] == 'WR'])}")
print(f"TE targets: {len(fasa_targets[fasa_targets['position'] == 'TE'])}")
print(f"\nMy roster size: {len(my_roster_droppable)}")
print(f"Drop candidates identified: {len(my_roster_droppable[my_roster_droppable['drop_recommendation'].str.contains('Consider', na=False)])}")
print(f"\nCurrent cap space: ${current_cap}")
print(f"Projected spend (primary bid): ${rb_targets.iloc[0]['suggested_bid_2yr']:.0f}" if len(rb_targets) > 0 else "N/A")